# Na minha primeira implementação, testei a versão do algoritmo em que não é realizada a triangulação de Delaunay antes
Em vez disso, computa-se a matriz de distâncias entre os pontos 

In [1]:
import numpy as np
from scipy.spatial.distance import pdist, squareform
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, FFMpegWriter
import random

class UnionFind:
    def __init__(self, n):
        """Inicializa estrutura de conjuntos disjuntos."""
        self.parent = list(range(n))

    def find(self, u):
        """Encontra o representante do conjunto de u com path compression."""
        if self.parent[u] != u:
            self.parent[u] = self.find(self.parent[u])
        return self.parent[u]

    def union(self, u, v):
        """Une os conjuntos de u e v."""
        ru, rv = self.find(u), self.find(v)
        if ru != rv:
            self.parent[rv] = ru
            return True
        return False

def generate_boruvka_frames(points, verbose=True):
    """Executa o algoritmo de Borůvka e gera os frames para animação."""
    n = len(points)
    dist_matrix = squareform(pdist(points))
    uf = UnionFind(n)
    mst_edges = []
    frames = []
    num_components = n

    while num_components > 1:
        cheapest = [None] * n
        for u in range(n):
            for v in range(n):
                if u != v and uf.find(u) != uf.find(v):
                    root = uf.find(u)
                    if cheapest[root] is None or dist_matrix[u][v] < dist_matrix[cheapest[root][0]][cheapest[root][1]]:
                        cheapest[root] = (u, v)

        new_edges = []
        for edge in cheapest:
            if edge is not None:
                u, v = edge
                if uf.find(u) != uf.find(v):
                    new_edges.append((u, v))

        frames.append((np.array(points), mst_edges.copy(), new_edges))

        for u, v in new_edges:
            if uf.union(u, v):
                mst_edges.append((u, v))
                num_components -= 1

    frames.append((np.array(points), mst_edges.copy(), []))

    if verbose:
        print("Arestas da Árvore Geradora Mínima (MST):")
        for u, v in mst_edges:
            print(f"{u} -- {v} (distância: {dist_matrix[u][v]:.4f})")

    return frames

def animate_boruvka(frames, filename="boruvka_mst.mp4"):
    """Cria e salva a animação da construção da MST."""
    fig, ax = plt.subplots()

    def update(frame):
        points, mst_edges, new_edges = frame
        ax.clear()
        ax.set_title(f"Etapa da MST (arestas: {len(mst_edges)})")
        ax.scatter(points[:, 0], points[:, 1], c='black')
        for u, v in mst_edges:
            ax.plot([points[u][0], points[v][0]], [points[u][1], points[v][1]], 'b-', linewidth=2)
        for u, v in new_edges:
            ax.plot([points[u][0], points[v][0]], [points[u][1], points[v][1]], 'r--', linewidth=1.5)
        ax.axis('equal')
        ax.grid(True)

    anim = FuncAnimation(fig, update, frames=frames, interval=1500)
    writer = FFMpegWriter(fps=1)
    anim.save(filename, writer=writer)
    plt.close(fig)


# Se estiver apenas testando o algoritmo, não precisa executar o bloco abaixo. É apenas uma demonstração de como a performance é pior sem Delaunay. Continue abaixo para a implementação melhor

In [2]:
random.seed(42)
def generate_random_points(num_points):
    return [(random.uniform(-10, 10), random.uniform(-10, 10)) for _ in range(num_points)]

# MUDE ESSAS VARIÁVEIS
points = generate_random_points(10000)
animated = False

import time
# Exemplo de uso
if __name__ == "__main__":
    np.random.seed(0)
    start_time = time.time()
    frames = generate_boruvka_frames(points, verbose=True)
    elapsed_time = time.time() - start_time
    print(f"Tempo de execução: {elapsed_time:.4f} segundos")

    if animated:
        animate_boruvka(frames, "boruvka_mst.mp4")

KeyboardInterrupt: 

A performance foi muito ruim para um número elevado de pontos (o algoritmo estava levando mais de 7 minutos). Vamos testar agora com Delaunay antes.

In [2]:
import numpy as np
from scipy.spatial import Delaunay, distance
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, FFMpegWriter
import random

class UnionFind:
    def __init__(self, n):
        self.parent = list(range(n))

    def find(self, u):
        if self.parent[u] != u:
            self.parent[u] = self.find(self.parent[u])
        return self.parent[u]

    def union(self, u, v):
        ru, rv = self.find(u), self.find(v)
        if ru != rv:
            self.parent[rv] = ru
            return True
        return False

def extract_edges_from_delaunay(points):
    """Extrai arestas únicas da triangulação de Delaunay."""
    tri = Delaunay(points)
    edges = set()
    for simplex in tri.simplices:
        for i in range(3):
            u, v = sorted((simplex[i], simplex[(i + 1) % 3]))
            edges.add((u, v))
    return list(edges)

def generate_boruvka_delaunay_frames(points, verbose=True):
    """Executa Borůvka usando apenas as arestas da triangulação de Delaunay."""
    points = np.array(points)
    n = len(points)
    edges = extract_edges_from_delaunay(points)
    dist = lambda u, v: np.linalg.norm(points[u] - points[v])
    uf = UnionFind(n)
    mst_edges = []
    frames = []
    num_components = n

    while num_components > 1:
        cheapest = [None] * n
        for u, v in edges:
            ru, rv = uf.find(u), uf.find(v)
            if ru != rv:
                if cheapest[ru] is None or dist(u, v) < dist(*cheapest[ru]):
                    cheapest[ru] = (u, v)
                if cheapest[rv] is None or dist(u, v) < dist(*cheapest[rv]):
                    cheapest[rv] = (u, v)

        new_edges = []
        seen = set()
        for edge in cheapest:
            if edge is not None:
                u, v = edge
                if uf.find(u) != uf.find(v):
                    if (u, v) not in seen and (v, u) not in seen:
                        new_edges.append((u, v))
                        seen.add((u, v))

        frames.append((np.array(points), mst_edges.copy(), new_edges))

        for u, v in new_edges:
            if uf.union(u, v):
                mst_edges.append((u, v))
                num_components -= 1

    frames.append((np.array(points), mst_edges.copy(), []))

    if verbose:
        print("Arestas da MST:")
        for u, v in mst_edges:
            print(f"{u} -- {v} (distância: {dist(u, v):.4f})")

    return frames

def animate_boruvka(frames, filename="boruvka_delaunay_mst.mp4"):
    """Gera animação da construção da MST com Delaunay ao fundo."""
    fig, ax = plt.subplots()

    # Extrai os pontos uma única vez (de qualquer frame) para calcular a triangulação
    base_points = frames[0][0]
    tri = Delaunay(base_points)

    def update(frame):
        points, mst_edges, new_edges = frame
        ax.clear()
        ax.set_title(f"MST parcial ({len(mst_edges)} arestas)")
        ax.scatter(points[:, 0], points[:, 1], c='black', zorder=3)

        # Desenha a triangulação de Delaunay em cinza claro
        for simplex in tri.simplices:
            for i in range(3):
                u, v = simplex[i], simplex[(i + 1) % 3]
                ax.plot(
                    [base_points[u][0], base_points[v][0]],
                    [base_points[u][1], base_points[v][1]],
                    color='lightgray', linestyle='-', linewidth=0.7, zorder=0
                )

        # Desenha as arestas da MST em azul
        for u, v in mst_edges:
            ax.plot([points[u][0], points[v][0]], [points[u][1], points[v][1]],
                    'b-', linewidth=2, zorder=2)

        # Desenha as novas arestas sendo consideradas em vermelho tracejado
        for u, v in new_edges:
            ax.plot([points[u][0], points[v][0]], [points[u][1], points[v][1]],
                    'r--', linewidth=1.5, zorder=1)

        ax.axis('equal')
        ax.grid(True)

    anim = FuncAnimation(fig, update, frames=frames, interval=1500)
    writer = FFMpegWriter(fps=1)
    anim.save(filename, writer=writer)
    plt.close(fig)


# Para testar o algoritmo, mude a váriavel "pts" abaixo e rode o algoritmo. 
Se quiser uma simulação visual com o matplotlib, mude a variável "animated" para "True". Se quiser gerar pontos aleatórios, utilize a função "generate_random_points" como pode ser visto abaixo. Caso contrário, passe uma array de pontos no formato [(a,b), (c,d), (e,f), ... (y,z)]

In [3]:
random.seed(42)
def generate_random_points(num_points):
    return [(random.uniform(-10, 10), random.uniform(-10, 10)) for _ in range(num_points)]

In [4]:
# MUDE ESSAS VARIÁVEIS
points = generate_random_points(10000)
animated = False

import time
# Exemplo de uso
if __name__ == "__main__":
    np.random.seed(0)
    start_time = time.time()
    frames = generate_boruvka_delaunay_frames(points, verbose=True)
    elapsed_time = time.time() - start_time
    print(f"Tempo de execução: {elapsed_time:.4f} segundos")
    if animated:
        animate_boruvka(frames, "boruvka_mst.mp4")

Arestas da MST:
0 -- 562 (distância: 0.0700)
1 -- 7325 (distância: 0.1457)
2 -- 6597 (distância: 0.1599)
3 -- 8142 (distância: 0.0675)
4 -- 4923 (distância: 0.0687)
5 -- 3051 (distância: 0.0953)
6 -- 9954 (distância: 0.0960)
7 -- 5625 (distância: 0.1656)
8 -- 9628 (distância: 0.1194)
9 -- 5194 (distância: 0.2088)
10 -- 1453 (distância: 0.0129)
11 -- 4394 (distância: 0.1710)
12 -- 3712 (distância: 0.1374)
13 -- 1298 (distância: 0.1425)
14 -- 277 (distância: 0.0868)
15 -- 6010 (distância: 0.1055)
16 -- 467 (distância: 0.0714)
17 -- 1075 (distância: 0.0197)
18 -- 7433 (distância: 0.2392)
19 -- 3153 (distância: 0.0225)
20 -- 4091 (distância: 0.1585)
21 -- 5230 (distância: 0.0968)
22 -- 5483 (distância: 0.0688)
23 -- 4710 (distância: 0.0384)
24 -- 4182 (distância: 0.0985)
25 -- 7778 (distância: 0.0279)
26 -- 129 (distância: 0.0766)
27 -- 1615 (distância: 0.0140)
28 -- 4188 (distância: 0.0932)
29 -- 4588 (distância: 0.0660)
30 -- 7223 (distância: 0.0702)
31 -- 7213 (distância: 0.1500)
32 -- 

A performance foi muito melhor com Delaunay antes